In [1]:
import sys
import time
import json
import numpy as np
import pandas as pd

import bjorn_support as bs
import onion_trees as ot
import mutations as bm
import visualize as bv
import reports as br
import data as bd

In [2]:
date = '2021-02-21'
unknown_val = 'None'

In [3]:
countries_fp = '/home/al/data/geojsons/gadm_countries.json'
states_fp = '/home/al/data/geojsons/gadm_divisions.json'
locations_fp = '/home/al/data/geojsons/gadm_locations.json'
subs = pd.read_csv(f'/valhalla/gisaid/subs_long_{date}.csv.gz', 
                   compression='gzip')
dels = pd.read_csv(f'/valhalla/gisaid/dels_long_{date}.csv.gz', 
                   compression='gzip')

/home/al/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (18) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
/home/al/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (25) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
# cols = ['mutation', 'strain', 'country', 'division', 'location', 'date_collected', 'absolute_coords', 'del_len']

In [5]:
dels['pos'] = dels['absolute_coords'].apply(lambda x: int(x.split(':')[0]))
dels['ref_codon'] = dels['del_seq'].copy()

In [6]:
dels['gene_start_pos'] = dels['gene'].apply(lambda x: bd.GENE2POS[x]['start']+2)

In [7]:
dels['pos_in_codon'] = (dels['pos'] - dels['gene_start_pos']) % 3

In [8]:
# Relative positions in codon counts [internal]
dels['pos_in_codon'].value_counts()

2    131881
0    112605
1     98030
Name: pos_in_codon, dtype: int64

In [9]:
def assign_deletion_codon_coords(x):
    if (x['pos_in_codon'] + x['del_len']) <= 3:
        return x['gene'] + ':DEL' + str(x['codon_num'] + (x['pos_in_codon']/3))
    deletion = x['gene'] + ':DEL' + str(x['codon_num'] + (x['pos_in_codon']/3))\
               + '/' + str(x['codon_num'] + (1 + (x['pos_in_codon']/3))\
               + (x['del_len']/3) - 1)
    return deletion

In [10]:
def assign_deletion(x):
    if (x['pos_in_codon'] + x['del_len']) <= 3:
        return x['gene'] + ':DEL' + str(x['codon_num'])
    deletion = x['gene'] + ':DEL' + str(x['codon_num']) + '/' + str(x['codon_num'] + (x['del_len']//3) - 1)
    return deletion

In [11]:
dels['mutation'] = dels[['pos_in_codon', 'gene', 'codon_num', 'del_len']].apply(assign_deletion, axis=1)
# dels['mutation'].value_counts()

In [12]:
dels['deletion_codon_coords'] = dels[['pos_in_codon', 'gene', 'codon_num', 'del_len']].apply(assign_deletion_codon_coords, axis=1)

In [13]:
# dels['mutation'] = dels['mutation'].apply(lambda x: x.split('.')[0])

In [14]:
def is_frameshift(x):
    if x % 3 == 0:
        return False
    return True
dels['is_frameshift'] = dels['del_len'].apply(is_frameshift)

In [15]:
print(subs.shape)
print(dels.shape)
subs['type'] = 'substitution'
muts = pd.concat([subs, dels])
print(muts.shape)

(10368708, 26)
(342516, 34)
(10711224, 38)


In [20]:
muts['country'] = muts['country'].astype(str)
muts['country_lower'] = muts['country'].str.lower()
muts['country_normed'] = muts['country_normed'].astype(str)
muts['country_normed_lower'] = muts['country_normed'].str.lower()
muts['division'] = muts['division'].astype(str)
muts['division_lower'] = muts['division'].str.lower()
muts['division_normed'] = muts['division_normed'].astype(str)
muts['division_normed_lower'] = muts['division_normed'].str.lower()
muts['location'] = muts['location'].astype(str)
muts['location_lower'] = muts['location'].str.lower()
muts['location_normed'] = muts['location_normed'].astype(str)
muts['location_normed_lower'] = muts['location_normed'].str.lower()

In [21]:
muts['country_normed'].value_counts()

United Kingdom                      5229292
United States                       1882498
Denmark                              936762
Switzerland                          263012
Japan                                254614
                                     ...   
Saint Vincent and the Grenadines         11
Cuba                                     11
Uzbekistan                                7
Andorra                                   4
belgium                                   3
Name: country_normed, Length: 158, dtype: int64

In [22]:
# muts.loc[muts['pangolin_lineage'].isin(['B.1.427'])].sort_values('date', ascending=True)[['idx', 'date', 'date_submitted', 'division', 'originating_lab']]

In [23]:
muts.columns

Index(['idx', 'replacements', 'pos', 'gene', 'gene_start_pos', 'codon_num',
       'codon_start', 'ref_codon', 'alt_codon', 'ref_aa', 'alt_aa', 'mutation',
       'strain', 'covsurver_prot_mutations', 'location', 'pangolin_lineage',
       'date_collected', 'accession_id', 'pangolin_version', 'clade',
       'date_submitted', 'country', 'division', 'country_normed',
       'division_normed', 'location_normed', 'type', 'seq_len',
       'del_positions', 'del_len', 'relative_coords', 'absolute_coords',
       'del_seq', 'prev_5nts', 'next_5nts', 'pos_in_codon',
       'deletion_codon_coords', 'is_frameshift', 'country_lower',
       'country_normed_lower', 'division_lower', 'division_normed_lower',
       'location_lower', 'location_normed_lower'],
      dtype='object')

In [24]:
muts['tmp'] = muts['date_collected'].str.split('-')
muts = muts[muts['tmp'].str.len()>=2]
muts.loc[muts['tmp'].str.len()==2, 'date_collected'] += '-15'
muts['date_collected'] = pd.to_datetime(muts['date_collected'], errors='coerce')
muts = muts[muts['date_collected']<date]

In [25]:
# muts.loc[muts['tmp'].str.len()==2, 'date']

In [26]:
muts['date_collected'] = muts['date_collected'].astype(str)

In [27]:
# muts['date'].max()

In [28]:
muts['date_collected'].max()

'2021-02-19'

In [29]:
test = muts.groupby('date_collected').agg(num_samples=('strain', 'nunique')).reset_index()
test[test.duplicated(subset=['date_collected'])]

,date_collected,num_samples


In [30]:
muts['date_collected']

29        2020-03-23
30        2020-03-23
31        2020-03-23
32        2020-03-23
33        2020-03-23
             ...    
342511    2021-02-11
342512    2021-02-06
342513    2021-02-06
342514    2021-02-08
342515    2020-02-18
Name: date_collected, Length: 10579337, dtype: object

In [31]:
with open(countries_fp) as f:
    countries = json.load(f)
muts['country_id'] = muts['country_normed'].apply(lambda x: countries.get(x, unknown_val))
with open(states_fp) as f:
    states = json.load(f)
muts['division_id'] = muts['division_normed'].apply(lambda x: states.get(x, unknown_val))
with open(locations_fp) as f:
    locations = json.load(f)
muts['location_id'] = muts['location_normed'].apply(lambda x: locations.get(x, unknown_val))

In [65]:
muts.rename(columns={
#     'date': 'date_collected',
#     'GISAID_clade': 'gisaid_clade',
#     'Nextstrain_clade': 'nextstrain_clade',
    'country': 'country_original',
    'division': 'division_original',
    'location': 'location_original',
    'country_lower': 'country_original_lower',
    'division_lower': 'division_original_lower',
    'location_lower': 'location_original_lower',
    'country_normed': 'country',
    'division_normed': 'division',
    'location_normed': 'location',
    'country_normed_lower': 'country_lower',
    'division_normed_lower': 'division_lower',
    'location_normed_lower': 'location_lower',
    'del_len': 'change_length_nt'
    }, inplace=True)

In [66]:
muts.columns

Index(['idx', 'replacements', 'pos', 'gene', 'gene_start_pos', 'codon_num',
       'codon_start', 'ref_codon', 'alt_codon', 'ref_aa', 'alt_aa', 'mutation',
       'strain', 'covsurver_prot_mutations', 'location_original',
       'pangolin_lineage', 'date_collected', 'accession_id',
       'pangolin_version', 'clade', 'date_submitted', 'country_original',
       'division_original', 'country', 'division', 'location', 'type',
       'seq_len', 'del_positions', 'change_length_nt', 'relative_coords',
       'absolute_coords', 'del_seq', 'prev_5nts', 'next_5nts', 'pos_in_codon',
       'deletion_codon_coords', 'is_frameshift', 'country_original_lower',
       'country_lower', 'division_original_lower', 'division_lower',
       'location_original_lower', 'location_lower', 'tmp', 'country_id',
       'division_id', 'location_id', 'date_modified', 'is_synonymous'],
      dtype='object')

In [67]:
# muts['nt_map_coords'] = muts[['gene', 'pos']].apply(bs.compute_acc_nt_pos, 
#                             args=(bd.GENE2NTCOORDS,), 
#                             axis=1)

In [68]:
# muts['aa_map_coords'] = muts[['gene', 'codon_num']].apply(bs.compute_acc_aa_pos, 
#                             args=(bd.GENE2AACOORDS,), 
#                             axis=1)

In [69]:
muts['date_modified'] = date

In [70]:
muts['is_synonymous'] = False
muts.loc[muts['ref_aa']==muts['alt_aa'], 'is_synonymous'] = True

In [71]:
muts.loc[muts['location']=='unk', 'location'] = unknown_val
# muts.loc[muts['purpose_of_sequencing']=='?', 'purpose_of_sequencing'] = unknown_val
# muts.loc[muts['genbank_accession']=='?', 'genbank_accession'] = unknown_val

In [72]:
muts.fillna(unknown_val, inplace=True)

In [73]:
muts.loc[muts['division']==muts['country'], 'division'] = unknown_val

In [74]:
sample_ids = muts[['strain']].drop_duplicates().sample(10)['strain'].unique()
test = muts[muts['strain'].isin(sample_ids)]

In [75]:
print(muts.shape)
# muts.drop_duplicates(subset=['strain', 'mutation'], inplace=True)
print(muts.shape)

(10579337, 50)
(10579337, 50)


In [76]:
muts['date_collected']

29        2020-03-23
30        2020-03-23
31        2020-03-23
32        2020-03-23
33        2020-03-23
             ...    
342511    2021-02-11
342512    2021-02-06
342513    2021-02-06
342514    2021-02-08
342515    2020-02-18
Name: date_collected, Length: 10579337, dtype: object

In [77]:
# muts.loc[muts['country']==muts['division']]

In [78]:
muts.columns

Index(['idx', 'replacements', 'pos', 'gene', 'gene_start_pos', 'codon_num',
       'codon_start', 'ref_codon', 'alt_codon', 'ref_aa', 'alt_aa', 'mutation',
       'strain', 'covsurver_prot_mutations', 'location_original',
       'pangolin_lineage', 'date_collected', 'accession_id',
       'pangolin_version', 'clade', 'date_submitted', 'country_original',
       'division_original', 'country', 'division', 'location', 'type',
       'seq_len', 'del_positions', 'change_length_nt', 'relative_coords',
       'absolute_coords', 'del_seq', 'prev_5nts', 'next_5nts', 'pos_in_codon',
       'deletion_codon_coords', 'is_frameshift', 'country_original_lower',
       'country_lower', 'division_original_lower', 'division_lower',
       'location_original_lower', 'location_lower', 'tmp', 'country_id',
       'division_id', 'location_id', 'date_modified', 'is_synonymous'],
      dtype='object')

In [96]:
meta_info = [
        'strain', 'accession_id',
        'date_modified', 'date_collected', 'date_submitted',
        'country_id', 'country', 'country_original', 'country_lower', 'country_original_lower',
        'division_id', 'division', 'division_original', 'division_lower', 'division_original_lower',
        'location_id', 'location', 'location_original', 'location_lower', 'location_original_lower',
#         'submitting_lab', 'originating_lab',
#         'authors', 
        'pangolin_lineage', 'pangolin_version',
        'clade', 
#         'nextstrain_clade',
#         'gisaid_epi_isl', 'genbank_accession',
#         'purpose_of_sequencing',
            ]

muts_info = ['type', 'mutation', 'gene', 
             'ref_codon', 'pos', 'alt_codon', 
             'is_synonymous', 
             'ref_aa', 'codon_num', 'alt_aa', 
             'absolute_coords', 
             'change_length_nt', 'is_frameshift',
             'deletion_codon_coords']

In [80]:
# # TEST 
# start = time.time()
# (test.groupby(meta_info, as_index=True)
#              .apply(lambda x: x[muts_info].to_dict('records'))
#              .reset_index()
#              .rename(columns={0:'mutations'})
#              .to_json(f'test_data/TEST_api_data_model_{date}.json', 
#                       orient='records',
# #                       compression='gzip'
#                      )
# )
# end = time.time()
# print(f'Execution time: {end - start} seconds')

In [81]:
# test||

In [82]:
# muts.drop(columns=['nt_map_coords', 'aa_map_coords'], inplace=True)

In [83]:
muts['date_submitted'].max()

'2021-02-21'

In [84]:
muts['date_submitted']

29        2020-04-17
30        2020-04-17
31        2020-04-17
32        2020-04-17
33        2020-04-17
             ...    
342511    2021-02-21
342512    2021-02-21
342513    2021-02-21
342514    2021-02-21
342515    2021-02-21
Name: date_submitted, Length: 10579337, dtype: object

In [85]:
muts['date_collected']

29        2020-03-23
30        2020-03-23
31        2020-03-23
32        2020-03-23
33        2020-03-23
             ...    
342511    2021-02-11
342512    2021-02-06
342513    2021-02-06
342514    2021-02-08
342515    2020-02-18
Name: date_collected, Length: 10579337, dtype: object

In [86]:
# muts['date_modified']

In [87]:
muts[muts['type']=='deletion']['mutation'].value_counts()

S:DEL69/70            97176
ORF1a:DEL3675/3677    87996
S:DEL144/144          86166
ORF1a:DEL1/25         11126
ORF8:DEL121            5014
                      ...  
ORF7a:DEL48/760           1
ORF7a:DEL63/83            1
ORF1a:DEL2777/2781        1
S:DEL849                  1
ORF7a:DEL75/80            1
Name: mutation, Length: 4784, dtype: int64

In [88]:
muts.loc[(muts['type']=='deletion')&(muts['mutation']==1)][['pos', 'change_length_nt', 'mutation']]

,pos,change_length_nt,mutation


In [89]:
# muts.loc[muts['change_length_nt']=='None', 'change_length_nt'] = -1
# muts['change_length_nt'] = muts['change_length_nt'].astype(int)

In [90]:
cols = ['absolute_coords', 'change_length_nt']
# muts[muts['change_length_nt']>=20000].sort_values('change_length_nt', ascending=False)[cols]

In [91]:
# muts[muts['change_length_nt']>=5000].sort_values('change_length_nt', ascending=False)['gene'].value_counts()

In [92]:
# muts[muts['change_length_nt']>=20000].sort_values('change_length_nt', ascending=False).shape

In [93]:
cols = ['ref_codon', 'pos', 'alt_codon', 'ref_aa', 'codon_num', 'alt_aa']
# muts[muts['mutation']=='S:N501N'][cols]

In [94]:
muts['country'].value_counts()

United Kingdom                      5148421
United States                       1881231
Denmark                              936762
Switzerland                          263012
Japan                                243281
                                     ...   
Saint Vincent and the Grenadines         11
Cuba                                     11
Uzbekistan                                7
Andorra                                   4
belgium                                   3
Name: country, Length: 156, dtype: int64

In [95]:
muts['country_lower'].value_counts()

united kingdom                      5148421
united states                       1881231
denmark                              936762
switzerland                          263012
japan                                243281
                                     ...   
saint-barthélemy                         12
cuba                                     11
saint vincent and the grenadines         11
uzbekistan                                7
andorra                                   4
Name: country_lower, Length: 155, dtype: int64

In [ ]:
# GENERATE JSON DATA MODEL
start = time.time()
(muts.groupby(meta_info, as_index=True)
             .apply(lambda x: x[muts_info].to_dict('records'))
             .reset_index()
             .rename(columns={0:'mutations'})
             .to_json(f'/valhalla/gisaid/api_data.json.gz',
                      orient='records',
                      compression='gzip'))
end = time.time()
print(f'Execution time: {end - start} seconds')

In [42]:
muts.drop_duplicates(subset=['strain', 'mutation'], inplace=True)

In [65]:
muts = muts[muts['gene']!='5UTR']